# Natural Language Processing

by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)
/ [GitHub](https://github.com/Hvass-Labs/TensorFlow-Tutorials) / [Videos on YouTube](https://www.youtube.com/playlist?list=PL9Hr9sNUjfsmEu1ZniY0XpHSzl5uihcXZ)

Modified by uramoon@kw.ac.kr

## Introduction

감성 분석 (Sentiment Analysis)은 자연어 처리 (Natural Language Processing, NLP)를 이용하여 텍스트의 감정을 파악하는 기술이다. 이 노트북에서는 영화 리뷰가 긍정적인지 부정적인지 분류할 것이다.

"This movie is not very good."을 보면 "very good"은 긍정적인 감정이지만 "not"이 있기 때문에 부정적인 감정으로 분류되어야 한다. 이러한 것을 어떻게 학습시킬 수 있을까?

1. 인공 신경망은 숫자를 입력으로 받아들이는데 텍스트를 어떻게 수치 데이터로 변환할지 생각해봐야 한다.
2. 문서의 길이는 문서마다 다른데 크기가 각기 다른 입력 데이터를 인공 신경망에 어떻게 입력해야할지 생각해봐야 한다.

## Flowchart

1. Tokenizer를 이용하여 각 단어를 정수로 변환한다. 예) the: 1, and: 2, a: 3, ...
2. 임베딩 (embedding)을 통해 각 정수를 n차원 벡터로 변환한다. (가까운 단어는 가깝게 위치)
3. 문서를 순환 신경망 (recurrent neural network, RNN)에 입력하여 0 (부정적)부터 1 (긍적적) 사이의 실수를 출력하게 훈련한다.

The flowchart of the algorithm is roughly:

<img src="https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/images/20_natural_language_flowchart.png?raw=1" alt="Flowchart NLP" style="width: 300px;"/>

## Recurrent Neural Network (RNN)

RNN의 기본 유닛은 Recurrent Unit (RU)으로 LSTM (Long-Short-Term-Memory)과 성능 하락을 최소화하면서 LSTM을 단순화한 GRU (Gated Recurrent Unit)가 많이 사용된다.

RU는 과거의 상태를 기억하고 있다가 현재의 입력에 대해 자신의 상태를 바꾸면서 값을 출력한다.

새로운 상태는 과거의 상태와 현재의 입력에 따라 결정된다. 예를 들어 최근의 입력에 "not"이 있었고, 현재의 입력이 "good"이라면 새로운 상태는 "not good"에 해당하는 부정적인 감정을 기억할 것이다.

![Recurrent unit](https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/images/20_recurrent_unit.png?raw=1)

### Unrolled Network

RNN에서 RU가 다음 입력에 과거의 정보를 전달하는 과정을 다음과 같이 펼쳐서 도식화할 수 있다.

RU의 메모리는 0으로 초기화된다.

가장 처음 "this"가 입력되면 메모리는 새로운 상태를 저장하고 무언가를 출력하지만 출력값을 사용하지는 않는다. 글을 끝까지 읽었을 때의 출력값만 활용할 것이다.

두 번째 단어는 "is"인데 바로 전에 읽은 "this"와 결합하여 새로운 상태를 저장한다.

세 번째 단어는 "not"인데 "not"을 기억하고 있다가 나중에 "good"을 읽었을 때 그것을 부정적인 단어로 바꿔주는 역할을 할 것이다.

문서의 마지막 단어를 읽었을 때 0부터 1사이의 값을 출력하는데 이를 활용하여 감성 분석을 수행한다.

Note that for the sake of clarity, this figure doesn't show the mapping from text-words to integer-tokens and embedding-vectors, as well as the fully-connected Sigmoid layer on the output.

![Unrolled network](https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/images/20_unrolled_flowchart.png?raw=1)

### Exploding & Vanishing Gradients

매 스텝 새로운 단어가 들어올 때마다 내부 상태를 변경할 경우 기울기 소실 혹은 폭주 문제가 발생할 수 있습니다.

하나의 텍스트가 500개의 단어로 구성된다고 했을 때 내부 상태가 500번 변화하게 되는데 각 변화마다 기울기를 곱할 때 기울기가 1 미만이면 그 값이 0에 가까워지고, 기울기가 1보다 크면 그 값은 매우 커집니다.

이러한 기울기 소실 / 폭주 문제를 해결하기 위해 매 입력마다 상태가 변화하지 않는 능력을 지닌 장기기억 메모리를 도입한 것이 LSTM과 GRU입니다.

## 준비 단계
1. 런타임을 GPU로 설정해주세요.
2. imdb.py와 download.py 파일을 Colab 환경에 복사해주세요.

## Imports

In [21]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

We need to import several things from Keras.

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Load Data

IMDB에서 50,000개의 영화 리뷰를 다운받습니다. Keras에 정제된 IMDB 데이터셋이 있지만 단어를 정수로 변환하는 작업을 직접 수행하기 위해 정제되지 않은 데이터셋을 사용합니다. 50,000개 중 25,000개는 훈련 데이터, 나머지 25,000개는 테스트 데이터입니다.



In [23]:
import imdb

Automatically download and extract the files.

In [24]:
imdb.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


## 다운받은 파일 확인
폴더를 새로고침하면 다운로드 받은 data 폴더가 보입니다.

1. 훈련 데이터 폴더: data/IMDB/aclImdb/train/
2. 테스트 데이터 폴더: data/IMDB/aclImdb/test/

In [25]:
# 훈련 데이터의 pos 폴더에서 10점짜리 긍정적 리뷰 확인
#  This movie gets better each time I see it (which is quite often).
!cat data/IMDB/aclImdb/train/pos/10001_10.txt

Brilliant over-acting by Lesley Ann Warren. Best dramatic hobo lady I have ever seen, and love scenes in clothes warehouse are second to none. The corn on face is a classic, as good as anything in Blazing Saddles. The take on lawyers is also superb. After being accused of being a turncoat, selling out his boss, and being dishonest the lawyer of Pepto Bolt shrugs indifferently "I'm a lawyer" he says. Three funny words. Jeffrey Tambor, a favorite from the later Larry Sanders show, is fantastic here too as a mad millionaire who wants to crush the ghetto. His character is more malevolent than usual. The hospital scene, and the scene where the homeless invade a demolition site, are all-time classics. Look for the legs scene and the two big diggers fighting (one bleeds). This movie gets better each time I see it (which is quite often).

In [26]:
# 훈련 데이터의 neg 폴더에서 1점짜리 부정적 리뷰 확인
# this story is too painful to watch.
!cat data/IMDB/aclImdb/train/neg/10002_1.txt

Sorry everyone,,, I know this is supposed to be an "art" film,, but wow, they should have handed out guns at the screening so people could blow their brains out and not watch. Although the scene design and photographic direction was excellent, this story is too painful to watch. The absence of a sound track was brutal. The loooonnnnng shots were too long. How long can you watch two people just sitting there and talking? Especially when the dialogue is two people complaining. I really had a hard time just getting through this film. The performances were excellent, but how much of that dark, sombre, uninspired, stuff can you take? The only thing i liked was Maureen Stapleton and her red dress and dancing scene. Otherwise this was a ripoff of Bergman. And i'm no fan f his either. I think anyone who says they enjoyed 1 1/2 hours of this is,, well, lying.

## 훈련 데이터와 테스트 데이터 만들기
x 값에는 리뷰 내용이 (텍스트), y 값에는 0 (부정) 혹은 1 (긍정)이 기록됩니다.


In [27]:
x_train_text, y_train = imdb.load_data(train=True)
x_test_text, y_test = imdb.load_data(train=False)

# Convert to numpy arrays.
y_train = np.array(y_train)
y_test = np.array(y_test)

In [28]:
print("Train-set size: ", len(x_train_text))
print("Test-set size:  ", len(x_test_text))

Train-set size:  25000
Test-set size:   25000


In [29]:
# TODO: 훈련데이터의 0번째 x값(리뷰)과 y값(정답)을 출력해보세요.
print(x_train_text[0])
print(y_train[0])

Probably the first Portuguese film I have seen in my life, and I enjoyed it. The plot is related of how the young army officers took the power in Portugal in 1974, to finally defeat the fascist government of Caetano and to also finalize the wars in the colonies, i.e. Mozambique, Angola, and Guinea (Bissau)- Cape Vert. Most of the events shown in the film reflect with exactitude the behavior of the army officers and soldiers to conduct the coup, of the oppressed people, who were very happy with this new development and the liberty, the resistance of Caetano's men, and also in a subtle way of most conservative officials, including Spinola, who took over as the new president. The Portuguese revolution can be remembered because of the action of several young officers, but for me the most interesting part of the film was when the young captain expressed that Portugal should develop itself democratically, and this is what the country achieved some years after this coup or revolution. The fil

## Tokenizer

인공신경망은 숫자를 입력받기 때문에 우선 각 단어를 tokenizer를 통해 정수로 변환합니다.<br>Tokenizer는 데이터셋에서 가장 많이 등장하는 n개의 단어를 정수로 변환합니다.<br> 예) the: 1, and: 2, a: 3, ...
<br>
일단 10000개의 단어만 사용하여 모델을 만들어봅시다.

In [30]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)

훈련 데이터와 테스트 데이터에 들어 있는 모든 단어를 변환할 필요가 있기 때문에 훈련 데이터와 테스트 데이터를 함께 tokenizer에 입력합니다.

In [31]:
%%time
data_text = x_train_text + x_test_text
tokenizer.fit_on_texts(data_text)

CPU times: user 10.1 s, sys: 61.6 ms, total: 10.2 s
Wall time: 10.5 s


Tokenizer가 찾아낸 많이 등장하는 단어들입니다. 해당 단어들은 정수로 변환되고 나머지 단어들은 제거될 것입니다.

In [32]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'are': 23,
 'his': 24,
 'have': 25,
 'be': 26,
 'one': 27,
 'he': 28,
 'all': 29,
 'at': 30,
 'by': 31,
 'an': 32,
 'they': 33,
 'so': 34,
 'who': 35,
 'from': 36,
 'like': 37,
 'or': 38,
 'just': 39,
 'her': 40,
 'out': 41,
 'about': 42,
 'if': 43,
 "it's": 44,
 'has': 45,
 'there': 46,
 'some': 47,
 'what': 48,
 'good': 49,
 'when': 50,
 'more': 51,
 'very': 52,
 'up': 53,
 'no': 54,
 'time': 55,
 'my': 56,
 'even': 57,
 'would': 58,
 'she': 59,
 'which': 60,
 'only': 61,
 'really': 62,
 'see': 63,
 'story': 64,
 'their': 65,
 'had': 66,
 'can': 67,
 'me': 68,
 'well': 69,
 'were': 70,
 'than': 71,
 'much': 72,
 'we': 73,
 'bad': 74,
 'been': 75,
 'get': 76,
 'do': 77,
 'great': 78,
 'other': 79,
 'will': 80,
 'also': 81,
 'into': 82,
 'p

Tokenizer로 훈련 데이터의 텍스트를 정수로 변환합니다.

In [33]:
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)

For example, here is a text from the training-set:

In [34]:
# TODO: 훈련 데이터의 0번째 원본 텍스트(x_train_text) 출력해보기
print(x_train_text[0])

Probably the first Portuguese film I have seen in my life, and I enjoyed it. The plot is related of how the young army officers took the power in Portugal in 1974, to finally defeat the fascist government of Caetano and to also finalize the wars in the colonies, i.e. Mozambique, Angola, and Guinea (Bissau)- Cape Vert. Most of the events shown in the film reflect with exactitude the behavior of the army officers and soldiers to conduct the coup, of the oppressed people, who were very happy with this new development and the liberty, the resistance of Caetano's men, and also in a subtle way of most conservative officials, including Spinola, who took over as the new president. The Portuguese revolution can be remembered because of the action of several young officers, but for me the most interesting part of the film was when the young captain expressed that Portugal should develop itself democratically, and this is what the country achieved some years after this coup or revolution. The fil

In [36]:
# TODO: 위에서 정수로 변환한 0번째 텍스트 출력해보기
# first는 86, of는 4, all은 29로 변환됐는지 살펴보세요.
print(x_train_tokens[0])

[239, 1, 86, 7658, 19, 10, 25, 107, 8, 56, 114, 2, 10, 511, 9, 1, 111, 6, 2469, 4, 85, 1, 186, 1248, 3854, 550, 1, 660, 8, 8, 5628, 5, 415, 4399, 1, 7659, 1310, 4, 2, 5, 81, 1, 1698, 8, 1, 10, 932, 2, 7888, 7364, 88, 4, 1, 706, 606, 8, 1, 19, 4423, 16, 1, 1932, 4, 1, 1248, 3854, 2, 1246, 5, 9498, 1, 7479, 4, 1, 83, 35, 70, 52, 673, 16, 11, 168, 979, 2, 1, 7365, 1, 7366, 4, 345, 2, 81, 8, 3, 1284, 95, 4, 88, 4679, 7540, 585, 35, 550, 121, 14, 1, 168, 1998, 1, 7658, 2938, 67, 26, 2025, 84, 4, 1, 205, 4, 439, 186, 3854, 18, 15, 68, 1, 88, 218, 173, 4, 1, 19, 13, 50, 1, 186, 1622, 4881, 12, 142, 2238, 392, 2, 11, 6, 48, 1, 679, 3225, 47, 153, 100, 11, 7479, 38, 2938, 1, 19, 81, 276, 12, 1, 1248, 3854, 2, 1246, 110, 461, 5, 504, 250, 57, 1, 88, 599, 4180, 70, 4780, 30, 1, 127]


We also need to convert the texts in the test-set to tokens.

In [37]:
# TODO: tokenizer로 전체 테스트 데이터를 정수로 변환해보세요.
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

## Padding and Truncating Data

순환신경망은 임의의 길이를 지닌 입력을 처리할 수 있지만 훈련의 속도를 높이기 위해 동일 길이를 지닌 입력 데이터의 묶음 (batch) 단위로 훈련이 이루어지는 관계로 편의상 모든 입력 데이터의 길이를 동일하게 합니다. 모든 입력 데이터의 길이를 동일하게 하는 방법은 크게 두 가지가 있습니다.

1. 가장 길이가 긴 리뷰의 길이에 맞추어 다른 리뷰들의 앞, 혹은 뒤에 공백을 삽입한다.

2. 적당한 길이를 정한다음 길이가 긴 리뷰는 앞, 혹은 뒤를 잘라내고 짧은 리뷰는 앞, 혹은 뒤에 공백을 삽입한다.

첫 번째 방법은 지나치게 긴 리뷰가 있을 경우 다른 리뷰들에 지나치게 많은 공백이 들어가 데이터셋이 커져 훈련하는데 오래 걸리고 메모리의 낭비도 심해지는 단점이 있어 우리는 두 번째 방법을 사용합니다.

In [38]:
# 각 리뷰에 단어 (토큰)가 몇 개씩 들어있는지 세어봅시다.

num_tokens = [] # 빈 리스트 생성

# TODO: num_tokens에 각 리뷰의 단어 숫자를 저장하세요.
# num_tokens 원소는 50,000개이며 처음 25,000은 훈련 데이터의 단어숫자가 저장되어 있고,
# 나머지 25,000은 테스트 데이터의 단어숫자가 저장되어 있습니다.
# Hint: x_train_tokens[0]은 첫 번째 훈련 데이터의 단어들이 저장되어 있습니다. 그 길이는 len 함수로 알 수 있습니다.
for i in range(len(x_train_tokens)):
  num_tokens.append(len(x_train_tokens[i]))



In [39]:
# TODO: num_tokens 리스트를 num_tokens NumPy 배열로 변환하세요.
num_tokens = np.array(num_tokens)

The average number of tokens in a sequence is:

In [40]:
# 모든 리뷰에서 사용한 평균 단어의 수는 다음과 같습니다.
np.mean(num_tokens)

223.73656

The maximum number of tokens in a sequence is:

In [41]:
# TODO: 가장 긴 리뷰의 단어수를 출력해보세요.
# Hint: np.max
np.max(num_tokens)

2142

In [42]:
# 리뷰에 사용된 단어수(토큰수)에 따른 리뷰의 수 살펴보기
# 대부분의 리뷰가 500개 이하의 토큰만 사용했음을 알 수 있음
plt.hist(num_tokens, bins=50)
plt.xlabel('Number of tokens')
plt.ylabel('Number of reviews')
plt.show()

The max number of tokens we will allow is set to the average plus 2 standard deviations.

In [43]:
# TODO: num_tokens에서 평균 + 2 * 표준편차를 구하여 우리가 허용할 최대 단어의 개수인 max_tokens를 설정해보세요.
# num_tokens가 정규분포를 따른다면 97% 이상이 max_tokens보다 적은 단어만을 사용할 것입니다.
# Hint: np.mean과 np.std 사용

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

551

This covers about 95% of the data-set.

In [44]:
# max_tokens보다 짧은 리뷰의 비율을 출력하는 코드
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.94508

리뷰의 94% 이상은 max_tokens보다 짧으므로 공백을 삽입하여 길이를 max_tokens로 맞춰주고, max_tokens보다 긴 리뷰의 길이는 max_tokens로 자를 필요가 있습니다. 모든 리뷰를 가장 길었던 리뷰의 길이로 맞췄다면 데이터셋 크기가 굉장히 커졌을 것입니다.<br>

공백을 삽입할 때 리뷰의 앞에 ('pre') 삽입할 수도 있고 마지막 ('post')에 삽입할 수도 있는데 인공신경망에 혼란을 주지 않기 위해 일관된 방식으로 삽입해야 합니다. (자르기도 동일)

In [45]:
# 공백을 리뷰의 앞에 삽입
pad = 'pre'

In [46]:
# 훈련 데이터의 길이를 max_tokens로 맞춰주기
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,  # 입력의 최대 길이는 max_tokens
                            padding=pad, truncating=pad)        # 패딩과 자르기 모두 리뷰의 앞부분에서 수행

In [47]:
# TODO: 테스트 데이터의 길이를 max_tokens로 맞춰주세요.
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens, padding=pad, truncating=pad)

We have now transformed the training-set into one big matrix of integers (tokens) with this shape:

In [48]:
# 훈련 데이터: (리뷰 개수, 각 리뷰의 길이)
x_train_pad.shape

(25000, 551)

The matrix for the test-set has the same shape:

In [49]:
# 테스트 데이터: (리뷰 개수, 각 리뷰의 길이)
x_test_pad.shape

(25000, 551)

For example, we had the following sequence of tokens above:

In [50]:
# 패딩 전의 0번째 훈련 데이터
np.array(x_train_tokens[0])

array([ 239,    1,   86, 7658,   19,   10,   25,  107,    8,   56,  114,
          2,   10,  511,    9,    1,  111,    6, 2469,    4,   85,    1,
        186, 1248, 3854,  550,    1,  660,    8,    8, 5628,    5,  415,
       4399,    1, 7659, 1310,    4,    2,    5,   81,    1, 1698,    8,
          1,   10,  932,    2, 7888, 7364,   88,    4,    1,  706,  606,
          8,    1,   19, 4423,   16,    1, 1932,    4,    1, 1248, 3854,
          2, 1246,    5, 9498,    1, 7479,    4,    1,   83,   35,   70,
         52,  673,   16,   11,  168,  979,    2,    1, 7365,    1, 7366,
          4,  345,    2,   81,    8,    3, 1284,   95,    4,   88, 4679,
       7540,  585,   35,  550,  121,   14,    1,  168, 1998,    1, 7658,
       2938,   67,   26, 2025,   84,    4,    1,  205,    4,  439,  186,
       3854,   18,   15,   68,    1,   88,  218,  173,    4,    1,   19,
         13,   50,    1,  186, 1622, 4881,   12,  142, 2238,  392,    2,
         11,    6,   48,    1,  679, 3225,   47,  1

This has simply been padded to create the following sequence. Note that when this is input to the Recurrent Neural Network, then it first inputs a lot of zeros. If we had padded 'post' then it would input the integer-tokens first and then a lot of zeros. This may confuse the Recurrent Neural Network.

In [51]:
# TODO: 패딩 후의 0번째 훈련 데이터를 출력해보세요.
# Hint: 바로 위와 다른 형태로 나와야 합니다.
np.array(x_train_pad[0])

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

## Tokenizer Inverse Map

정수의 배열을 다시 텍스트로 바꿔주는 함수를 만들어봅시다.

In [52]:
idx = tokenizer.word_index # tokenizer는 단어 (key)-> 정수 (value)이므로
inverse_map = dict(zip(idx.values(), idx.keys())) # 정수 (value) -> 단어 (key)로 보내면 됩니다.

Helper-function for converting a list of tokens back to a string of words.

In [53]:
# tokens_to_string 함수에 정수 배열을 입력하면 텍스트로 출력됩니다.
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]

    # Concatenate all words.
    text = " ".join(words)

    return text

For example, this is the original text from the data-set:

In [54]:
# 훈련 데이터의 0번째 리뷰 실제 텍스트
x_train_text[0]

"Probably the first Portuguese film I have seen in my life, and I enjoyed it. The plot is related of how the young army officers took the power in Portugal in 1974, to finally defeat the fascist government of Caetano and to also finalize the wars in the colonies, i.e. Mozambique, Angola, and Guinea (Bissau)- Cape Vert. Most of the events shown in the film reflect with exactitude the behavior of the army officers and soldiers to conduct the coup, of the oppressed people, who were very happy with this new development and the liberty, the resistance of Caetano's men, and also in a subtle way of most conservative officials, including Spinola, who took over as the new president. The Portuguese revolution can be remembered because of the action of several young officers, but for me the most interesting part of the film was when the young captain expressed that Portugal should develop itself democratically, and this is what the country achieved some years after this coup or revolution. The fi

In [55]:
# 정수로 변환된 훈련 데이터의 0번째 리뷰
np.array(x_train_tokens[0])

array([ 239,    1,   86, 7658,   19,   10,   25,  107,    8,   56,  114,
          2,   10,  511,    9,    1,  111,    6, 2469,    4,   85,    1,
        186, 1248, 3854,  550,    1,  660,    8,    8, 5628,    5,  415,
       4399,    1, 7659, 1310,    4,    2,    5,   81,    1, 1698,    8,
          1,   10,  932,    2, 7888, 7364,   88,    4,    1,  706,  606,
          8,    1,   19, 4423,   16,    1, 1932,    4,    1, 1248, 3854,
          2, 1246,    5, 9498,    1, 7479,    4,    1,   83,   35,   70,
         52,  673,   16,   11,  168,  979,    2,    1, 7365,    1, 7366,
          4,  345,    2,   81,    8,    3, 1284,   95,    4,   88, 4679,
       7540,  585,   35,  550,  121,   14,    1,  168, 1998,    1, 7658,
       2938,   67,   26, 2025,   84,    4,    1,  205,    4,  439,  186,
       3854,   18,   15,   68,    1,   88,  218,  173,    4,    1,   19,
         13,   50,    1,  186, 1622, 4881,   12,  142, 2238,  392,    2,
         11,    6,   48,    1,  679, 3225,   47,  1

We can recreate this text except for punctuation and other symbols, by converting the list of tokens back to words:

In [56]:
# TODO: 정수 형태의 0번째 리뷰를 다시 단어로 변환해보세요.
# 모두 소문자로 변경되고, 잘 쓰이지 않는 단어들은 삭제된 상태입니다.
# Hint: 위에서 정의한 tokens_to_string 함수 이용
tokens_to_string(x_train_tokens[0])

'probably the first portuguese film i have seen in my life and i enjoyed it the plot is related of how the young army officers took the power in in 1974 to finally defeat the fascist government of and to also the wars in the i e and guinea cape most of the events shown in the film reflect with the behavior of the army officers and soldiers to conduct the coup of the people who were very happy with this new development and the liberty the resistance of men and also in a subtle way of most conservative officials including who took over as the new president the portuguese revolution can be remembered because of the action of several young officers but for me the most interesting part of the film was when the young captain expressed that should develop itself and this is what the country achieved some years after this coup or revolution the film also shows that the army officers and soldiers never wanted to kill anyone even the most serious enemies were respected at the end'

## Create the Recurrent Neural Network

We are now ready to create the Recurrent Neural Network (RNN). We will use the Keras API for this because of its simplicity. See Tutorial #03-C for a tutorial on Keras.

In [57]:
model = Sequential()

자연어 처리를 위한 순환 신경망의 가장 첫 번째 층은 임베딩층입니다. 우리가 선택한 10000개의 단어를 우리가 지정한 차원에 보내 비슷한 단어가 가까운 곳에 위치하도록 만들 것입니다. (Tokenizer가 만든 정수는 1: the, 2: and, 3: a, 4: of과 같이 뜻이 비슷하지도 않은데 단어들이 가까운 곳에 위치해 있습니다.)

일반적으로 임베딩 차원은 100에서 300 정도를 사용하는데 일단 8차원만 사용해봅시다.

In [58]:
embedding_size = 8

The embedding-layer also needs to know the number of words in the vocabulary (`num_words`) and the length of the padded token-sequences (`max_tokens`). We also give this layer a name because we need to retrieve its weights further below.

In [59]:
# 임베딩층 추가
model.add(Embedding(input_dim=num_words,      # 사용하는 단어의 개수
                    output_dim=embedding_size,# 임베딩 차원
                    input_length=max_tokens,  # 리뷰의 길이
                    name='layer_embedding'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


순환 유닛인 GRU를 추가하고 Dense로 묶어 하나의 출력에서 0과 1사이의 실수를 출력하도록 합니다.

In [60]:
model.add(GRU(units=256))
model.add(Dense(1, activation='sigmoid')) # 이전 층의 입력을 받아들여 0부터 1사이의 실수값 하나 출력 (sigmoid)

Use the Adam optimizer with the given learning-rate.

In [61]:
optimizer = Adam(learning_rate=1e-3)

Compile the Keras model so it is ready for training.

In [62]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [63]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ layer_embedding (Embedding)          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Train the Recurrent Neural Network

We can now train the model. Note that we are using the data-set with the padded sequences. We use 5% of the training-set as a small validation-set, so we have a rough idea whether the model is generalizing well or if it is perhaps over-fitting to the training-set.

In [64]:
# validation을 통해 테스트셋에서의 성능을 가늠할 수 있습니다.
# 훈련데이터에 대한 성능은 개선되는데 validation에서 악화된다면 과적합이 일어나는 것일 수도 있습니다.
from keras.callbacks import EarlyStopping

# TODO: 자유롭게 설정, restore_best_weights=True 추천
es = EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True,verbose=1)

model.fit(x_train_pad, y_train, callbacks=es,
          validation_split=0.2, epochs=10000, batch_size=64)

Epoch 1/10000
313/313 ━━━━━━━━━━━━━━━━━━━━ 17s 38ms/step - accuracy: 0.6303 - loss: 0.6557 - val_accuracy: 0.7654 - val_loss: 0.5876
Epoch 2/10000
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 38ms/step - accuracy: 0.8353 - loss: 0.3851 - val_accuracy: 0.7834 - val_loss: 0.5309
Epoch 3/10000
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 38ms/step - accuracy: 0.8861 - loss: 0.2825 - val_accuracy: 0.8342 - val_loss: 0.4840
Epoch 4/10000
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.9086 - loss: 0.2345 - val_accuracy: 0.8192 - val_loss: 0.4639
Epoch 5/10000
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.9323 - loss: 0.1879 - val_accuracy: 0.7922 - val_loss: 0.5595
Epoch 6/10000
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.9477 - loss: 0.1486 - val_accuracy: 0.8542 - val_loss: 0.4069
Epoch 7/10000
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.9539 - loss: 0.1310 - val_accuracy: 0.8004 - val_loss: 0.6126
Epoch 8/10000
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - accuracy: 

## Performance on Test-Set

Now that the model has been trained we can calculate its classification accuracy on the test-set.

In [65]:
# 테스트셋에서의 성능 평가
%%time
result = model.evaluate(x_test_pad, y_test)

782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.8570 - loss: 0.3778
CPU times: user 7.45 s, sys: 315 ms, total: 7.76 s
Wall time: 9.33 s


## New Data

Let us try and classify new texts that we make up. Some of these are obvious, while others use negation and sarcasm to try and confuse the model into mis-classifying the text.

In [66]:
# 새로운 데이터셋 8개 생성하여 texts에 저장
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Not a good movie!"
text8 = "This movie really sucks! Can I get my money back please?"
texts = [text1, text2, text3, text4, text5, text6, text7, text8]

We first convert these texts to arrays of integer-tokens because that is needed by the model.

In [67]:
# TODO: tokenizer로 texts를 정수로 변환하세요.
tokens = tokenizer.texts_to_sequences(texts)

To input texts with different lengths into the model, we also need to pad and truncate them.

In [72]:
# TODO: 패딩으로 tokens의 길이를 맞춰주세요.
tokens_pad = pad_sequences(tokens, maxlen=max_tokens, padding=pad, truncating=pad)
tokens_pad.shape

(8, 551)

We can now use the trained model to predict the sentiment for these texts.

In [70]:
# TODO: model로 tokens_pad를 예측해보세요.
# 1에 가까울 수록 긍정적이고 0에 가까울 수록 부정적입니다.
# 잘 예측했나요? 운에 따라 결과가 다를 수 있습니다.
model.predict(tokens_pad)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


array([[0.98931515],
       [0.999418  ],
       [0.9549755 ],
       [1.        ],
       [0.7566327 ],
       [0.9816062 ],
       [0.9893595 ],
       [0.06901944]], dtype=float32)

A value close to 0.0 means a negative sentiment and a value close to 1.0 means a positive sentiment. These numbers will vary every time you train the model.

## Newer Data
우리 모델은 평균 200개 이상의 단어가 쓰인 리뷰로 학습됐습니다. 위와 같이 매우 짧은 리뷰에 대해서는 잘 동작하지 않을 수도 있으니 <br>IMDb에서 듄 Part Two의 10점짜리 리뷰와 5점 이하의 리뷰로 테스트 해봅시다.<br>
https://www.imdb.com/title/tt15239678/reviews

In [73]:
text1 = "Had the pleasure to watch this film in an early screening and was completely blown away. Dune: Part 2 is everything one could ask for from a film of its kind. As a big fan of the Game of Thrones franchise, it's been a long time since iv'e encountered this level of world-building and epicness. The plot and story development are carried out in an awe-inspiring manner throughout the movie, progressing at a precise pace toward a spectacular climax that is executed perfectly. Denis Villeneuve continues to prove himself as one of the most promising filmmakers of our time, and if it was up to me I would keep him in these high-budget epic tales such as these since there are very few directors working today that can tackle this genre as good as he does. The film received praise from many great filmmakers, the most notable being Christopher Nolan (The Dark Knight Trilogy, Oppenheimer), who very accurately compared Villeneuve's achievement in this film to the Empire Strikes Back of the modern era. Would highly recommend to re-watch the first movie in order to appreciate the subtleties and foreshadowing better, though it's not completely necessary since the movie is pretty self-contained. More specifically, one of the most prominent and impressive strengths of the movie is how it stands on its own without relying too heavily on the first part. Of course, for those who really want to enjoy the story, there's no real point in watching the second part without having seen the first one. However, even those who watched it a few years ago and don't remember every little plot detail will be able to catch up very quickly while watching the Pt. 2 and will understand all the important details-even the smaller ones. Non-ideally, even someone who hasn't watched the first movie at all will greatly enjoy this one, as it is very self-contained despite relying heavily on the first movie-an impressive and highly commendable screenplay effort by the writers. Timothée Chalamet delivers a very impressive and charismatic performance, to the point where it seems like this is the role he was born to play. Zendaya also gives a very good performance, with significantly more screen time than in the first movie. Austin Butler manages to be terrifying in the role of the villain, Rebecca Ferguson shines in one of the best performances of the year, and Javier Bardem surprises in a role that sometimes provides the comic relief of the movie. In addition to them, the film is filled with top-tier actors and actresses who all perform their roles amazingly. Hans Zimmer's score is masterful just as you'd expect, and one step up from the Oscar winning and Grammy nominated score of Pt. 1. Technical aspects such as VFX, Production Design, Sound, Editing, etc. Are all top notch and awards-worthy. The action sequences are absolutely mind blowing and sent chills down my spine. Denis direction is impeccable, and the story is absolutely fascinating- continuing to develop characters from Pt. 1 even deeper and introduce new incredible and intriguing characters. In my personal opinion, the movie is better than the first part in pretty much every aspect. While the first part was excellent, it mainly served as a setup and positioning of the pieces for the sequel. In Part 2, the story reaches its climax, with one of the strongest climaxes seen on the big screen in recent decades, befitting the complex work Frank Herbert wrote many years ago. I believe (and surely hope) this movie will be a major player in the next Award Season. Below-the-line wins are pretty much guaranteed as of now, but I hope it will get some love in above-the-line categories such as Direction and even Best Picture, perhaps to break stigmas presented against the Fantasy/Sci-Fi genre in recent years at the Academy. As a big fan of the genre, unless a better competitor will be released later this eligibility period, maybe it's time for a movie like this to triumph once again. While the movie serves as a great wrap-up to the story introduced in Pt. 1, in my opinion it could serve as an even better set-up to a possible masterpiece in Dune: Messiah, which I really hope will get green-lit soon. Don't miss the opportunity to catch this movie on IMAX, since I believe it's an historic piece of epic-Fantasy/Sci-Fi cinema and a movie that will be remembered as a classic of the genre. Extremely recommended."
text2 = "If you liked or loved the first one, the same will apply for this one. Personally, I loved this one even more and I think general audiences will as well. So I hope it does well at the box office because I need Dune Messiah now. This is everything I love about going to the movies. Also, Hans Zimmer.For book readers, I'd say this one takes more liberties than the first, but they were changes I liked, some for the better even. For the most part, it still hits the main beats from the book with a few things altered. The biggest change being no time jump. Therefore, a certain character doesn't fully appear in the movie contrary to the book. Though, they still use the character in a different way that I liked personally. I know it's early in the year, but you can lock some Oscar nominations for this movie: Best Picture, Best Director, Best Adapted Screenplay, Best Editing, Best Cinematography, Best Score, Best Sound, Best Visual Effects, Best Costume, Best Production Design, Best Makeup, and hopefully Rebecca Ferguson this time for Best Supporting Actress. I guess that's pretty much every category now that I've typed it out lol."
text3 = "This is one of those films that is overrated by a whole 5 stars because people who don't get it or fall asleep blame themselves rather than the film. Their inclination is to follow the herd... or the emperor in this case. Only problem is that the emperor has no clothes. It's a cultural phenomena whereby if something seems or has the pretence of being intelligent, half the population is scared to call it out as dumb. It's just a bunch of nonsense art-house type scenes thrown together without any particular order or purpose.. and they could do some decent CGI because, thanks to the abundance of sand, there is really very little on screen at any one time. There's really nothing else here and nothing that works."
text4 = "It's the worst movie I've watched in a long time. If someone who never saw a previous 1984 film or read a novel watches this one would not understand a thing because there is no logical explanation between some of the events like when Paul's sister appeared all of a sudden and no explanation how or why. The movie itsef is very slow and boring. The main actor looks like a whiney teen aristocrat instead of a strong young man. Even physically he doesn't look like someone who can stand a real fight or lead the whole nation. In the novel Paul Atreides is extremely disciplined and confident in every thing he does. All of his actions and words are deliberate and precisely controlled, and it is this that makes him the super human being - Kwisatz Haderaca. The spice awakens his abilities but doesn't transform him into someone else. Total waste of money and time."
texts = [text1, text2, text3, text4]

In [74]:
#TODO: 상기 texts에 대해 예측해보세요.
#Hint: tokenizer와 padding을 이용해야 합니다.
tokens = tokenizer.texts_to_sequences(texts)
tokens_pad = pad_sequences(tokens, maxlen=max_tokens, padding=pad, truncating=pad)
model.predict(tokens_pad)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


array([[0.9988418 ],
       [0.9893713 ],
       [0.00709718],
       [0.11264417]], dtype=float32)

## Embeddings

임베딩을 통해 각 단어가 어떻게 변환되는지 살펴봅시다.

The model cannot work on integer-tokens directly, because they are integer values that may range between 0 and the number of words in our vocabulary, e.g. 10000. So we need to convert the integer-tokens into vectors of values that are roughly between -1.0 and 1.0 which can be used as input to a neural network.

This mapping from integer-tokens to real-valued vectors is also called an "embedding". It is essentially just a matrix where each row contains the vector-mapping of a single token. This means we can quickly lookup the mapping of each integer-token by simply using the token as an index into the matrix. The embeddings are learned along with the rest of the model during training.

Ideally the embedding would learn a mapping where words that are similar in meaning also have similar embedding-values. Let us investigate if that has happened here.

First we need to get the embedding-layer from the model:

In [75]:
# 임베딩 층 불러오기
layer_embedding = model.get_layer('layer_embedding')

We can then get the weights used for the mapping done by the embedding-layer.

In [76]:
# 임베딩 층의 가중치 불러오기
weights_embedding = layer_embedding.get_weights()[0]

Note that the weights are actually just a matrix with the number of words in the vocabulary times the vector length for each embedding. That's because it is basically just a lookup-matrix.

In [77]:
# 10,000개의 단어를 8차원으로 보내고 있습니다.
weights_embedding.shape

(10000, 8)

Let us get the integer-token for the word 'good', which is just an index into the vocabulary.

In [78]:
# tokenizer에서 good의 위치
token_good = tokenizer.word_index['good']
token_good

49

Let us also get the integer-token for the word 'great'.

In [79]:
# tokenizer에서 great의 위치
token_great = tokenizer.word_index['great']
token_great

78

These integertokens may be far apart and will depend on the frequency of those words in the data-set.

Now let us compare the vector-embeddings for the words 'good' and 'great'. Several of these values are similar, although some values are quite different. Note that these values will change every time you train the model.

In [80]:
# good의 임베딩 결과
weights_embedding[token_good]

array([ 0.06917246,  0.07500466,  0.02024258, -0.06345233, -0.07336847,
        0.04206818, -0.0595849 , -0.00975448], dtype=float32)

In [81]:
# great의 임베딩 결과
weights_embedding[token_great]

array([ 0.18121855,  0.14708585,  0.09529335, -0.15164873, -0.18549275,
        0.09806824, -0.15447125, -0.09425583], dtype=float32)

Similarly, we can compare the embeddings for the words 'bad' and 'horrible'.

In [82]:
# tokenizer에서 bad와 horrible
token_bad = tokenizer.word_index['bad']
token_horrible = tokenizer.word_index['horrible']

In [83]:
# TODO: bad의 임베딩 결과
weights_embedding[token_bad]

array([-0.07029106, -0.09501244, -0.12914671,  0.14653099,  0.14387043,
       -0.1074779 ,  0.09129538,  0.06694379], dtype=float32)

In [84]:
# TODO: horrible의 임베딩 결과
weights_embedding[token_horrible]

array([-0.13663626, -0.17742288, -0.18578404,  0.15125124,  0.17309251,
       -0.19713497,  0.19125336,  0.17514713], dtype=float32)

임베딩 결과를 보면 good과 great은 비슷한 곳에 위치하고 good과 bad는 반대되는 곳에 위치한 것을 볼 수 있습니다.

### Sorted Words

코사인 유사도로 임베딩 공간에서 비슷한 단어를 찾아봅시다.


In [85]:
# 주어진 단어와 비슷한 단어를 찾아주는 함수
def print_sorted_words(word, metric='cosine'):
    """
    Print the words in the vocabulary sorted according to their
    embedding-distance to the given word.
    Different metrics can be used, e.g. 'cosine' or 'euclidean'.
    """

    # Get the token (i.e. integer ID) for the given word.
    token = tokenizer.word_index[word]

    # Get the embedding for the given word. Note that the
    # embedding-weight-matrix is indexed by the word-tokens
    # which are integer IDs.
    embedding = weights_embedding[token]

    # Calculate the distance between the embeddings for
    # this word and all other words in the vocabulary.
    distances = cdist(weights_embedding, [embedding],
                      metric=metric).T[0]

    # Get an index sorted according to the embedding-distances.
    # These are the tokens (integer IDs) for words in the vocabulary.
    sorted_index = np.argsort(distances)

    # Sort the embedding-distances.
    sorted_distances = distances[sorted_index]

    # Sort all the words in the vocabulary according to their
    # embedding-distance. This is a bit excessive because we
    # will only print the top and bottom words.
    sorted_words = [inverse_map[token] for token in sorted_index
                    if token != 0]

    # Helper-function for printing words and embedding-distances.
    def _print_words(words, distances):
        for word, distance in zip(words, distances):
            print("{0:.3f} - {1}".format(distance, word))

    # Number of words to print from the top and bottom of the list.
    k = 10

    print("Distance from '{0}':".format(word))

    # Print the words with smallest embedding-distance.
    _print_words(sorted_words[0:k], sorted_distances[0:k])

    print("...")

    # Print the words with highest embedding-distance.
    _print_words(sorted_words[-k:], sorted_distances[-k:])

We can then print the words that are near and far from the word 'great' in terms of their vector-embeddings. Note that these may change each time you train the model.

In [86]:
# great와 비슷한 단어 10개, 반대 단어 10개
print_sorted_words('great', metric='cosine')

Distance from 'great':
0.000 - great
0.008 - symbolism
0.008 - enjoyed
0.009 - voight
0.009 - extraordinary
0.009 - lily
0.009 - surrender
0.010 - sissy
0.010 - deemed
0.011 - jackie
...
1.986 - disappointment
1.987 - mundane
1.988 - unfunny
1.989 - chick
1.990 - continually
1.990 - fiasco
1.991 - actor's
1.991 - dvd's
1.992 - fiennes
1.993 - contrived


Similarly, we can print the words that are near and far from the word 'worst' in terms of their vector-embeddings.

In [88]:
# TODO: worst와 비슷한 단어 10개, 반대 단어 10개
print_sorted_words('worst',metric='cosine')

Distance from 'worst':
0.000 - worst
0.003 - lindsay
0.003 - mercifully
0.004 - atrocious
0.004 - fodder
0.004 - jerky
0.004 - quantum
0.005 - unremarkable
0.005 - embarrassed
0.006 - ham
...
1.995 - excellent
1.996 - demille
1.996 - devotion
1.996 - gruff
1.996 - fascinating
1.996 - owns
1.996 - neglected
1.996 - byrne
1.997 - favorites
1.997 - quintessential


## Conclusion

감성 분석에서 순환 신경망은 비교적 만족스러운 성능을 보이지만 사람과는 전혀 다른 방식으로 감정을 계산해냅니다. 거대 언어 모델에 비하여 상당히 적은 자원만을 사용했기 때문에 성능이 좋게 나오지 않았으니 참고하세요.

## 연습문제 (수행할 필요 없습니다.)

수행할 때마다 다른 결과를 얻을 수 있습니다.

* 현재 Tokenizer가 가장 자주 쓰이는 10,000개의 단어를 처리하는데 5000개만 사용하면 성능이 어떻게 변할까요?
* 임베딩을 8차원에서 수행하는데 200차원으로 늘리면 성능이 어떻게 변할까요?


## License (MIT)

Copyright (c) 2022 by uramoon@kw.ac.kr<br>
Copyright (c) 2018 by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.